In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import v2
import numpy as np

In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 16
batch_size = 8
learning_rate = 0.001

transform = v2.Compose(
    [v2.ToTensor(),
     v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

tarin_loder = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified
[[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ...
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ...
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ...
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ...

 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ...
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ...
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ...
  [216 184 140]
  [151 118  84]
  [123  92  72]]]


In [11]:
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.conv1(x)
        x = self.pool(F.relu(x))  # -> n, 6, 14, 14
        x = self.conv2(x)
        x = self.pool(F.relu(x))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x

model = ConvNet()
model.to(device)
critertion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(tarin_loder)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(tarin_loder):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = critertion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print(f'Epoche [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
print("Finished training")


with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')


torch.Size([3, 32, 32])
tensor([[[0.4118, 0.4039, 0.4118,  ..., 0.4039, 0.4039, 0.4039],
         [0.4275, 0.4196, 0.4196,  ..., 0.4196, 0.4196, 0.4196],
         [0.4275, 0.4118, 0.4196,  ..., 0.4275, 0.4275, 0.4196],
         ...,
         [0.4824, 0.4588, 0.4588,  ..., 0.4667, 0.4667, 0.4667],
         [0.4824, 0.4588, 0.4667,  ..., 0.4902, 0.4667, 0.4667],
         [0.4824, 0.4667, 0.4667,  ..., 0.4824, 0.4745, 0.4667]],

        [[0.4118, 0.4039, 0.4118,  ..., 0.4039, 0.4039, 0.4039],
         [0.4275, 0.4196, 0.4196,  ..., 0.4196, 0.4196, 0.4196],
         [0.4275, 0.4118, 0.4196,  ..., 0.4275, 0.4275, 0.4196],
         ...,
         [0.4824, 0.4588, 0.4588,  ..., 0.4667, 0.4667, 0.4667],
         [0.4824, 0.4588, 0.4667,  ..., 0.4902, 0.4667, 0.4667],
         [0.4824, 0.4667, 0.4667,  ..., 0.4824, 0.4745, 0.4667]],

        [[0.4118, 0.4039, 0.4118,  ..., 0.4039, 0.4039, 0.4039],
         [0.4275, 0.4196, 0.4196,  ..., 0.4196, 0.4196, 0.4196],
         [0.4275, 0.4118, 0.4196, 

ZeroDivisionError: float division by zero